# Setup Environment

We need to install [interpret](https://github.com/interpretml/interpret/) and [gamchanger](https://github.com/interpretml/gam-changer/) packages.

In [2]:
# Install `interpretml` and `gamchanger` packages.
!pip install --upgrade interpret gamchanger

# Model Training

We will train a simple [EBM model](https://interpret.ml/docs/ebm.html) to predict if an indivisual's income is above 50K using the [census dataset](https://archive.ics.uci.edu/ml/datasets/census+income).

In [7]:
import pandas as pd
import numpy as np
import gamchanger as gc
import interpret

from json import load
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier

print('GAM Changer version: ', gc.__version__)
print('Interpret version: ', interpret.__version__)

df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    header=None)

df.columns = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"
]

train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == " <=50K" else 1) #Turning response into 0 and 1

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

GAM Changer version:  0.1.12
Interpret version:  0.4.2


In [2]:
ebm = ExplainableBoostingClassifier(random_state=seed, n_jobs=-1)
ebm.fit(X_train, y_train)

ExplainableBoostingClassifier(n_jobs=-1, random_state=1)

# GAM Changing

Then we can start to **investigate**, **validate**, and **edit** the trained EBM model using GAM Changer.

GAM Changer expects:

1. Trained EBM model
2. Sample data (optional)
    
GAM Changer uses the sample data to compute model performance, feature correlation, etc. We suggest always providing sample data, unless the data is sensitive and you plan to share GAM Changer visualizations/outputs with external collaborators.

We recommend to generate sample data using the **validation set** (if you have one), or a **large subset of the training set**. GAM Changer can support up to 10k+ data points (upper bound is the browser's memory limit), and the it can provide realtime feedback when the sample size is about less than 3k.


In [3]:
# Randomly sample 2000 points from the training set for GAM Changer
rand_indexes = np.random.choice(range(len(y_train)), 2000)

X_sample = X_train.to_numpy()[rand_indexes]
y_sample = y_train.to_numpy()[rand_indexes]

In [4]:
gc.visualize(ebm, X_sample, y_sample)

24it [00:00, 2912.37it/s]


<iframe
 srcdoc="<!DOCTYPE html><html lang="en"><head><meta charset='utf-8'><meta name='viewport' content='width = device-width, initial-scale = 1'><title>GAM Changer</title><style>html,body{position:relative;width:100%;height:100%}body{color:#333;margin:0;padding:0;box-sizing:border-box;font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Oxygen-Sans,Ubuntu,Cantarell,"Helvetica Neue",sans-serif}a{color:rgb(0,100,200);text-decoration:none}a:hover{text-decoration:underline}a:visited{color:rgb(0,80,160)}label{display:block}input,button,select,textarea{font-family:inherit;font-size:inherit;-webkit-padding:0.4em 0;padding:0.4em;margin:0 0 0.5em 0;box-sizing:border-box;border:1px solid #ccc;border-radius:2px}input:disabled{color:#ccc}</style><script defer src='...'></script><script defer src='...'></script></head><body></body></html>"
 frameBorder="0"
 width="100%"
 height="645px"
 id="gam-changer-iframe-16975657">

## Use the changed EBM

Suppose you have maded some edits on your EBM model, and save a `.gamchanger` file by clicking the save button on the botttom right. Let's load the new model in Python!

Unfortunately, we cannot automatically transfer the new model back to Python (restriction of computational notebooks for security concerns). Instead, we will use Python to load the downloaded `.gamchanger` file.

For cloud-based notebooks (e.g., Colab), you would need to upload the `.gamchanger` file to the working directory so that you can load it from your notebook.

For example, here we load a previous editing history `edit-6-14-2023.gamchanger` to the root directory on the left panel, then use the code in the below cell.

In [5]:
# Read from file    

# Use below code to load from `~/Downloads`
# gc_dict = load(open(os.path.join(os.path.expanduser('~'), 'Downloads/edit-6-14-2023.gamchanger'), 'r'))

# Use below code to load from `./`
gc_dict = load(open('./edit-6-14-2023.gamchanger', 'r'))

# gc.get_edited_model will return a copy of your original EBM where edits are applied
new_ebm = gc.get_edited_model(ebm, gc_dict)

new_ebm

ExplainableBoostingClassifier(n_jobs=-1, random_state=1)